In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import os
import datetime
pd.options.display.max_rows = 20

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

import warnings
warnings.filterwarnings(action='ignore')

## 아래는 matplotlib 한국어 표시해주는 코드

from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)

# 생활인구

## 코드파일 처리

In [2]:
dcode = pd.read_excel('./data/행정동코드.xlsx',encoding='cp949')
dcode = dcode.iloc[:,0:4]
dcode = dcode[dcode.시도명 =="서울특별시"]
dcode = dcode.drop_duplicates()
dcode.columns = ["code","시도명","구","동"]
dcode['code'] = dcode['code'].map(lambda x : str(x)[:-2])
dcode.head()

,code,시도명,구,동
0,11000000,서울특별시,NaN,NaN
1,11110000,서울특별시,종로구,NaN
2,11110515,서울특별시,종로구,청운효자동
12,11110530,서울특별시,종로구,사직동
24,11110540,서울특별시,종로구,삼청동


## 생활인구파일 : 일별로 합산

In [3]:
def people_count_sum(df):    
    df = df.reset_index().iloc[:,0:4]
    df.columns = ["날짜","hour","code","생활인구수"]
    #df.drop(['hour'], axis = 1,inplace = True)
    df_return = df.groupby(['날짜', 'code']).apply(func)
    df2 = df_return.reset_index()
    df2['code'] = df2['code'].map(lambda x : str(x))
    res = pd.merge(df2,dcode,on="code")
    res.drop(['code'],axis = 1, inplace = True)
    return res

In [4]:
def func(x):
    d={}
    d['생활인구수'] = (x['생활인구수'].sum())/24

    return pd.Series(d, index=['생활인구수'])

In [5]:
list_ = []
for k in range(2,9):
    data = pd.read_csv(f'./data/LOCAL_PEOPLE_DONG_20200{k}.csv',encoding ='utf-8')
    df = people_count_sum(data)
    list_.append(df)

In [6]:
#for k in range(2,9):
#    data = pd.read_csv(f'./data/LOCAL_PEOPLE_DONG_20190{k}.csv',encoding ='utf-8')
#    df = people_count_sum(data)
#    list_.append(df)

## '구별'로 묶기

In [7]:
def func_dong(x):
    d={}
    d['생활인구수'] = (x['생활인구수'].sum())

    return pd.Series(d, index=['생활인구수'])

In [8]:
res = pd.concat(list_)
res.head()

,날짜,생활인구수,시도명,구,동
0,20200201,367962.2054,서울특별시,종로구,청운효자동
1,20200202,357848.5414,서울특별시,종로구,청운효자동
2,20200203,384159.4535,서울특별시,종로구,청운효자동
3,20200204,388443.1466,서울특별시,종로구,청운효자동
4,20200205,396399.9476,서울특별시,종로구,청운효자동


In [9]:
res_return = res.groupby(['날짜', '구']).apply(func_dong)
res1 = res_return.reset_index()

## 저장

In [10]:
res.to_csv('생활인구_전처리.csv',encoding='cp949') #동포함

In [11]:
res1.to_csv('생활인구_전처리_구.csv',encoding='cp949') #구까지만

# 유동인구

In [12]:
def yd_sum(yd):
    yd.drop(['시간(1시간단위)','연령대(10세단위)','성별'],axis = 1, inplace = True)
    yd.columns = ["날짜","시","구","유동인구수"]
    yd_return = yd.groupby(['날짜', '구']).apply(func_yd)
    yd2 = yd_return.reset_index()
    return yd2

In [13]:
def func_yd(x):
    d={}
    d['유동인구수'] = (x['유동인구수'].sum())/24

    return pd.Series(d, index=['유동인구수'])

In [14]:
list_yd = []
for k in range(2,9):
    data = pd.read_csv(f'./data/Floating_Population_200{k}.csv',encoding ='utf-8')
    yd = yd_sum(data)
    list_yd.append(yd)

In [15]:
#for k in range(3,9):
#    data = pd.read_csv(f'./data/FLT_SEOUL_0{k}MONTH.csv',encoding='utf-8')
#    yd = yd_sum(data)
#    list_yd.append(yd)

In [16]:
res2 = pd.concat(list_yd)
res2

,날짜,구,유동인구수
0,20200201,강남구,12636840
1,20200201,강동구,8339770
2,20200201,강북구,5966500
3,20200201,강서구,10864880
4,20200201,관악구,9966300
...,...,...,...
770,20190831,용산구,5143930
771,20190831,은평구,7981020
772,20190831,종로구,4977300
773,20190831,중구,4532980


In [17]:
res2.to_csv('유동인구_전처리.csv',encoding='cp949')

# 생활/유동 합치기

In [23]:
sum_all = pd.merge(res1, res2, how='outer', on = ["날짜","구"])
sum_all

,날짜,구,생활인구수,유동인구수
0,20190201,강남구,2.045038e+07,NaN
1,20190201,강동구,1.181991e+07,NaN
2,20190201,강북구,7.271637e+06,NaN
3,20190201,강서구,1.338179e+07,NaN
4,20190201,관악구,1.187355e+07,NaN
...,...,...,...,...
10620,20200831,용산구,6.772697e+06,5104870.0
10621,20200831,은평구,1.079006e+07,7987210.0
10622,20200831,종로구,6.549824e+06,5137120.0
10623,20200831,중구,6.420853e+06,5528340.0


In [24]:
sum_all.isnull().sum()

날짜         0
구          0
생활인구수      0
유동인구수    725
dtype: int64

In [25]:
sum_all.to_csv('생활유동합침.csv',encoding='cp949')

# 코로나 확진자 추가

In [170]:
data1 = pd.read_csv('./data/코로나19발생현황.csv', encoding='cp949')
data2 = pd.read_csv('./data/FINAL_FINAL_코로나추가.csv', encoding='utf-8')

In [173]:
data = pd.merge(data2,data1,on="날짜",how="inner")
data.head()

,Unnamed: 0,날짜,구,생활인구수,유동인구수,전체점포수,프랜차이즈점포수,일반점포수,확진자수,신규,누적
0,0,2020-02-01,강남구,759286.0842,526535.0000,55875,4371,51504,0,0,0
1,1,2020-02-01,강동구,519147.6209,347490.4167,20974,1901,19073,0,0,0
2,2,2020-02-01,강북구,319527.8081,248604.1667,13417,1075,12342,0,0,0
3,3,2020-02-01,강서구,555865.8080,452703.3333,25902,2633,23269,0,0,0
4,4,2020-02-01,관악구,520445.2208,415262.5000,19734,1908,17826,0,0,0


In [ ]:
data.to_csv('FINAL_FINAL_코로나추가.csv',encoding='cp949')

In [175]:
covid = pd.read_csv('./data/코로나서울확진자.csv',encoding='cp949')
covid.tail(8)

,환자번호,감염경로,확진일,거주지,퇴원현황
5455,13,해외 접촉(우한 교민),2020-02-02,타시도,퇴원
5456,11,#6 접촉,2020-01-31,종로구,퇴원
5457,10,#6 접촉,2020-01-31,종로구,퇴원
5458,9,#5 접촉,2020-01-31,성북구,퇴원
5459,7,해외 접촉(중국 우한시),2020-01-30,마포구,퇴원
5460,6,#3 접촉,2020-01-30,종로구,퇴원
5461,5,해외 접촉(중국 우한시),2020-01-30,중랑구,퇴원
5462,2,해외 접촉(중국 우한시),2020-01-24,강서구,퇴원


In [156]:
def func_covid(x):
    d={}
    d['서울신규'] = (x['환자번호'].count())

    return pd.Series(d, index=['서울신규'])

In [157]:
covid.tail()

,환자번호,감염경로,확진일,거주지,퇴원현황
5458,9,#5 접촉,2020-01-31,성북구,퇴원
5459,7,해외 접촉(중국 우한시),2020-01-30,마포구,퇴원
5460,6,#3 접촉,2020-01-30,종로구,퇴원
5461,5,해외 접촉(중국 우한시),2020-01-30,중랑구,퇴원
5462,2,해외 접촉(중국 우한시),2020-01-24,강서구,퇴원


In [158]:
covid2 = covid.groupby(['확진일']).apply(func_covid)

In [159]:
covid2 = covid2.reset_index()
covid2.columns = ["날짜","서울신규"]

In [160]:
covid2["서울누적"] = covid2["서울신규"].cumsum()
covid2.tail()

,날짜,서울신규,서울누적
224,2020-10-03,16,5376
225,2020-10-04,19,5395
226,2020-10-05,18,5413
227,2020-10-06,30,5443
228,2020-10-07,20,5463


In [161]:
final_ = pd.read_csv('FINAL_FINAL_코로나추가.csv',encoding='cp949')
final_.tail()

,Unnamed: 0,Unnamed: 0.1,날짜,구,생활인구수,유동인구수,전체점포수,프랜차이즈점포수,일반점포수,확진자수,신규,누적
5334,5334,5334,2020-08-31,은평구,449585.9084,332800.4167,18350,1572,16778,3,248,19947
5335,5335,5335,2020-08-31,종로구,272909.3277,214046.6667,25318,1608,23710,2,248,19947
5336,5336,5336,2020-08-31,중구,267535.5238,230347.5000,40015,2062,37953,0,248,19947
5337,5337,5337,2020-08-31,중랑구,359603.2996,293332.0833,16180,1286,14894,6,248,19947
5338,5338,5338,2020-08-31,중랑구,359603.2996,293332.0833,16180,1286,14894,4,248,19947


In [176]:
final = pd.merge(final_,covid2,on="날짜", how="left")
final.tail()

,Unnamed: 0,Unnamed: 0.1,날짜,구,생활인구수,유동인구수,전체점포수,프랜차이즈점포수,일반점포수,확진자수,신규,누적,서울신규,서울누적
5334,5334,5334,2020-08-31,은평구,449585.9084,332800.4167,18350,1572,16778,3,248,19947,104.0,4019.0
5335,5335,5335,2020-08-31,종로구,272909.3277,214046.6667,25318,1608,23710,2,248,19947,104.0,4019.0
5336,5336,5336,2020-08-31,중구,267535.5238,230347.5000,40015,2062,37953,0,248,19947,104.0,4019.0
5337,5337,5337,2020-08-31,중랑구,359603.2996,293332.0833,16180,1286,14894,6,248,19947,104.0,4019.0
5338,5338,5338,2020-08-31,중랑구,359603.2996,293332.0833,16180,1286,14894,4,248,19947,104.0,4019.0


In [164]:
final.isnull().sum()

Unnamed: 0        0
Unnamed: 0.1      0
날짜                0
구                 0
생활인구수             0
유동인구수             0
전체점포수             0
프랜차이즈점포수          0
일반점포수             0
확진자수              0
신규                0
누적                0
서울신규            600
서울누적            600
dtype: int64

In [177]:
final["서울신규"] = final["서울신규"].fillna(0)
final["서울누적"] = final["서울누적"].fillna(method = 'ffill')
final["서울누적"] = final["서울누적"].fillna(0)
final.head(2)

,Unnamed: 0,Unnamed: 0.1,날짜,구,생활인구수,유동인구수,전체점포수,프랜차이즈점포수,일반점포수,확진자수,신규,누적,서울신규,서울누적
0,0,0,2020-02-01,강남구,759286.0842,526535.0000,55875,4371,51504,0,0,0,0.0,0.0
1,1,1,2020-02-01,강동구,519147.6209,347490.4167,20974,1901,19073,0,0,0,0.0,0.0


In [166]:
final.drop(['Unnamed: 0.1'],axis = 1, inplace = True)
final.tail()

,Unnamed: 0,날짜,구,생활인구수,유동인구수,전체점포수,프랜차이즈점포수,일반점포수,확진자수,신규,누적,서울신규,서울누적
5334,5334,2020-08-31,은평구,449585.9084,332800.4167,18350,1572,16778,3,248,19947,104.0,4019.0
5335,5335,2020-08-31,종로구,272909.3277,214046.6667,25318,1608,23710,2,248,19947,104.0,4019.0
5336,5336,2020-08-31,중구,267535.5238,230347.5000,40015,2062,37953,0,248,19947,104.0,4019.0
5337,5337,2020-08-31,중랑구,359603.2996,293332.0833,16180,1286,14894,6,248,19947,104.0,4019.0
5338,5338,2020-08-31,중랑구,359603.2996,293332.0833,16180,1286,14894,4,248,19947,104.0,4019.0


In [167]:
final.to_csv('FINAL_FINAL_코로나추가2.csv',encoding='cp949')

In [168]:
final.isnull().sum()

Unnamed: 0    0
날짜            0
구             0
생활인구수         0
유동인구수         0
전체점포수         0
프랜차이즈점포수      0
일반점포수         0
확진자수          0
신규            0
누적            0
서울신규          0
서울누적          0
dtype: int64